In [5]:
import numpy as np
import tensorflow as tf

from mnist import MNIST
import matplotlib.pyplot as plt

# Data Preprocessing

## Functions

In [22]:
def randomize(dataset, labels):
    permutation = np.random.permutation(labels.shape[0])
    shuffled_dataset = dataset[permutation, :, :]
    shuffled_labels = labels[permutation]
    return shuffled_dataset, shuffled_labels

def one_hot_encode(np_array):
    return (np.arange(10) == np_array[:,None]).astype(np.float32)

def reformat_data(dataset, labels, image_width, image_height, image_depth):
    np_dataset_ = np.array([np.array(image_data).reshape(image_width, image_height, image_depth) for image_data in dataset])
    np_labels_ = one_hot_encode(np.array(labels, dtype=np.float32))
    np_dataset, np_labels = randomize(np_dataset_, np_labels_)
    return np_dataset, np_labels

def flatten_tf_array(array):
    shape = array.get_shape().as_list()
    return tf.reshape(array, [shape[0], shape[1] * shape[2] * shape[3]])

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

## MNIST Data

In [6]:
mnist_folder = './data/mnist/'
mnist_image_width = 28
mnist_image_height = 28
mnist_image_depth = 1
mnist_num_labels = 10

mndata = MNIST(mnist_folder)

# mnist_train_dataset_: list of len 60000; eachelement list of 784 len; mnist_train_labels_: list of len 60000; eachelement one int
mnist_train_dataset_, mnist_train_labels_ = mndata.load_training()

# mnist_test_dataset_: list of len 10000; eachelement list of 784 len; mnist_test_labels_: list of len 10000; eachelement one int
mnist_test_dataset_, mnist_test_labels_ = mndata.load_testing()

In [43]:
# mnist_train_dataset: (60000, 28, 28, 1); mnist_train_labels: (60000, 10)
mnist_train_dataset, mnist_train_labels = reformat_data(mnist_train_dataset_, mnist_train_labels_, mnist_image_width, mnist_image_height, mnist_image_depth)

# mnist_test_dataset: (10000, 28, 28, 1); mnist_test_labels: (10000, 10)
mnist_test_dataset, mnist_test_labels = reformat_data(mnist_test_dataset_, mnist_test_labels_, mnist_image_width, mnist_image_height, mnist_image_depth)

# MLP Model

In [46]:
learning_rate = 0.1 # ??????????????????????
batch_size = 400 # 60000/400 = 150 batches
num_epochs = 50
n_input = mnist_image_width * mnist_image_height * mnist_image_depth # num of features: 784
n_classes = 10

## Define training inputs as placeholders, and weights and biases as variables

In [63]:
X = tf.placeholder("float", [None, n_input]) # for each batch: (400, 4116)
Y = tf.placeholder("float", [None, n_classes]) # for each batch: (400, 10)

# Weights and biases
n_hidden_1 = 10  # number of neurons in hidden layers
weights = {
	'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])), # 1st hidden layer: (784, 10)
	'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_1])), # 2nd hidden layer: (10, 10)
	'out': tf.Variable(tf.random_normal([n_hidden_1, n_classes])) # output layer: (10, 10)
}
biases = {
	'b1': tf.Variable(tf.random_normal([n_hidden_1])), # bias for the 1st hidden layer: (10,)
	'b2': tf.Variable(tf.random_normal([n_hidden_1])), # bias for the 2nd hidden layer: (10,)
	'b_out': tf.Variable(tf.random_normal([n_classes])) # bias for the output layer: (10,)
}

# Define layers

In [ ]:
def multilayer_perceptron(x):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_2 = tf.add(tf.matmul(layer_1, weights['out1']), biases['out2']) # Added layer
    out_layer = tf.nn.sigmoid(tf.matmul(layer_2, weights['out2']) + biases['out3'])
    return out_layer

In [ ]:
logits = multilayer_perceptron(X) # >>>(?, 2)
# Define loss(softmax_cross_entropy_with_logits) and optimizer(AdamOptimizer)
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# Initializing the variables
init = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    start_time = time.time()
    sess.run(init)

    h = h5py.File('data1.hdf5', 'r')
    n1 = h.get('data_train')
    n2 = h.get('data_test')

    # Training cycle
    total_batch_train = int(n1.shape[0] / batch_size) # 60:number of batches for train dataset
    print(total_batch_train)
    total_batch_test = int(n2.shape[0] / batch_size) # 2:number of batches for test dataset
    

    for iter_num in range(num_epochs):
        avg_acc_train = 0.
        avg_acc_test = 0.
        for i in range(total_batch_train):
            train_x = n1[(i) * batch_size: (i + 1) * batch_size, ...]
            train_y = labels_train[(i) * batch_size: (i + 1) * batch_size, :]

            _, c_train, _logits_train = sess.run([train_op, loss_op, logits], feed_dict={X: train_x, Y: train_y})
            _label_train = [np.argmax(i) for i in _logits_train]
            _label_train_y = [np.argmax(i) for i in train_y]
            _accuracy_train = np.mean(np.array(_label_train) == np.array(_label_train_y))
            avg_acc_train += _accuracy_train


        for j in range(total_batch_test):
            test_x = n2[(j) * batch_size: (j + 1) * batch_size, ...]
            test_y = labels_test[(j) * batch_size: (j + 1) * batch_size, :]

            c_test, _logits_test = sess.run([loss_op, logits], feed_dict={X: test_x, Y: test_y})
            _label_test = [np.argmax(i) for i in _logits_test]
            _label_test_y = [np.argmax(i) for i in test_y]
            _accuracy_test = np.mean(np.array(_label_test) == np.array(_label_test_y))
            avg_acc_test += _accuracy_test

        # print the train and test accuracies   
        print("Train acc: %f, Test_acc: %f" % (avg_acc_train/total_batch_train, avg_acc_test/total_batch_test))
    duration = time.time() - start_time
    print('Time elapsed - {} seconds.'.format(duration))

In [68]:
!jupyter nbconvert --output-dir='../html-output' MLP.ipynb --to html --output MLP.html

[NbConvertApp] Converting notebook MLP.ipynb to html
[NbConvertApp] Writing 306490 bytes to ../html-output\MLP.html
